## Analysis of Controller on Linear Simulation

This notebook generates the following tables and figures in the paper

> Yeh, Christopher, et al. "Online learning for robust voltage control under uncertain grid topology." _arXiv preprint arXiv:2306.16674_ (2023).

- Tables: 1 (top)
- Figures: 2a, 3, 5a, 7

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

In [ ]:
from __future__ import annotations

from glob import glob
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from analysis import calculate_violations, plot_error_and_etahat, plot_voltages
from utils import load_pkl, savefig

# hide top and right splines on plots
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

In [ ]:
plots_dir = 'plots/tsg_final/'
os.makedirs(plots_dir, exist_ok=True)

In [ ]:
from network_utils import (
    create_56bus,
    create_RX_from_net,
    read_load_data)

net = create_56bus()
R, X = create_RX_from_net(net, noise=0)  # true R and X
p, qe = read_load_data()  # in MW and MVar
T, n = p.shape
print(f'T={T}, n={n}')

v_nom = 12**2  # nominal squared voltage magnitude, units kV^2
v_sub = v_nom  # fixed squared voltage magnitude at substation, units kV^2

vpars = qe @ X + p @ R + v_sub  # shape [T, n]

Vpar_min = np.min(vpars, axis=0)  # shape [n]
Vpar_max = np.max(vpars, axis=0)  # shape [n]
Vpar = (Vpar_min, Vpar_max)

In [ ]:
print('max-voltage node:', np.argmax(vpars.max(axis=0)))
print('min-voltage node:', np.argmin(vpars.min(axis=0)))

## Figure 2a

linear simulation, no controller

In [ ]:
plot_voltages(vpars, ylim=(11.0, 13.4), yticks=None,
              plots_dir=plots_dir, filename='linear_nocontrol',
              legend_filename='buses_legend')

## Table 1, top

linear simulation, mistakes and violations

In [ ]:
pkl_paths = {
    ('known', None): 'out/CBCconst_δ20_η10_20230810_011115.pkl',  # fixed X̂, learned etahat
}
for seed in [8, 9, 10, 11]:
    pkl_paths |= {
        # default: δ=20
        ('unknown', seed): glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_2*.pkl')[0],
        ('topo-14', seed): glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knowntopo14_2*.pkl')[0],
        ('lines-14', seed): glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
    }

pkls = {}
for (name, seed), pkl_path in pkl_paths.items():
    pkl = load_pkl(pkl_path)
    pkls[(name, seed)] = pkl
    print(f'{name: <15} {str(seed): <4}', pkl.keys())

In [ ]:
figs_and_axs = {}
for seed in [8, 9, 10, 11]:
    figs_and_axs[seed] = plt.subplots(1, 3, figsize=(9, 3))

rows = []
for key, pkl in pkls.items():
    info, seed = key
    ax = None
    if seed in figs_and_axs:
        fig, axs = figs_and_axs[seed]
        if key[0] == 'unknown': ax = axs[0]
        elif key[0] == 'topo-14': ax = axs[1]
        elif key[0] == 'lines-14': ax = axs[2]

    num_mistakes, avg_viol, max_viol = calculate_violations(
        key=key, pkl=pkl, ax=ax, T=T, n=n)
    rows.append((info, seed, num_mistakes, avg_viol, max_viol))

# df.groupby().agg('std') gives sample standard deviation (ddof=1), which is what we want
df = pd.DataFrame.from_records(rows, columns=['info', 'seed', 'mistakes', 'avg_viol', 'max_viol'])
stats = df.groupby(['info'])[['mistakes', 'avg_viol', 'max_viol']].agg(['mean', 'std'])

with pd.option_context('display.precision', 1):
    display(stats)
with pd.option_context('display.precision', 2):
    display(stats)

## Figure 3

linear simulation, voltage curves and model error

In [ ]:
pkl_paths = {
    ('known', None): 'out/CBCconst_δ20_η10_20230810_011115.pkl',  # fixed X̂, learned etahat
}
seeds = [8]  # [8, 9, 10, 11]
for seed in seeds:
    pkl_paths |= {
        # default: δ=20
        ('unknown', seed): glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_2*.pkl')[0],
        ('topo-14', seed): glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knowntopo14_2*.pkl')[0],
        ('lines-14', seed): glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
    }

pkls = {}
for (name, seed), pkl_path in pkl_paths.items():
    pkl = load_pkl(pkl_path)
    pkls[(name, seed)] = pkl
    print(f'{name: <15} {str(seed): <4}', pkl.keys())

In [ ]:
def fig3abcd(seed: int) -> None:
    for name in ['unknown', 'topo-14', 'lines-14']:
        data = pkls[(name, seed)]
        filename = f'linear_{name}_s{seed}'
        plot_voltages(data['vs'], plots_dir=plots_dir, filename=filename)
    
    data = pkls[('known', None)]
    filename = f'linear_known'
    plot_voltages(data['vs'], plots_dir=plots_dir, filename=filename)


fig3abcd(seed=8)

In [ ]:
def fig3e(seed: int) -> None:
    fig3e_pkls = {
        name: pkls[(name, seed)]
        for name in ['unknown', 'topo-14', 'lines-14']
    }
    fig3e_pkls['known'] = pkls[('known', None)]
    plot_error_and_etahat(
        fig3e_pkls, plots_dir=plots_dir, filename=f'linear_error_s{seed}',
        legend_loc='top')


fig3e(seed=8)

## Figure 5a

linear simulation, effect of $\delta$ on consistent model chasing

In [ ]:
pkl_paths = {}
seeds = [8]  # [8, 9, 10, 11]
for seed in seeds:
    pkl_paths |= {
        (r'η* known', seed): glob(f'out/CBCproj_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        (r'δ=1', seed): glob(f'out/CBCproj_δ1_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        (r'δ=20', seed): glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        (r'δ=100', seed): glob(f'out/CBCproj_δ100_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        (r'δ=500', seed): glob(f'out/CBCproj_δ500_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
    }

pkls = {}
for (name, seed), pkl_path in pkl_paths.items():
    pkl = load_pkl(pkl_path)
    pkls[(name, seed)] = pkl
    print(f'{name: <15} {str(seed): <4}', pkl.keys())

In [ ]:
def fig5a(seed: int, legend_loc: str | None) -> None:
    pkls_by_delta = {
        name: pkls[(name, seed)]
        for name in ['η* known', 'δ=1', 'δ=20', 'δ=100', 'δ=500']
    }
    filename = f'linear_error_by_delta_s{seed}'
    plot_error_and_etahat(
        pkls_by_delta, plots_dir=plots_dir, filename=filename,
        legend_loc=legend_loc, etamax=10)


fig5a(seed=8, legend_loc='separate')

## Figure 7

linear simulation, detecting topology change

In [ ]:
seed = 8
pkl_paths = {
    # default: δ=20
    'unknown': glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_topochange_2*.pkl')[0],
    'topo-14': glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knowntopo14_topochange_2*.pkl')[0],
    'lines-14': glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_topochange_2*.pkl')[0],
}

pkls = {}
for name, pkl_path in pkl_paths.items():
    pkl = load_pkl(pkl_path)
    pkls[name] = pkl
    print(f'{name: <15}', pkl.keys())

    pkl['dists1'] = pkl['dists']
    pkl['dists'] = {
        'X_true': np.concatenate([pkl['dists1']['X_true'], pkl['dists2']['X_true']]),
        'η': np.concatenate([pkl['dists1']['η'], pkl['dists2']['η']]),
        't': np.concatenate([pkl['dists1']['t'], np.array(pkl['dists2']['t']) + int(T/2)]),
    }

In [ ]:
plot_error_and_etahat(
    pkls, plots_dir=plots_dir,
    filename=f'linear_detectchange_s{seed}',
    legend_loc='separate')

## Unused

In [ ]:
pkl_paths = {
    # ('known', None): 'out/CBCconst_20230809_234150.pkl',  # fixed X̂, fixed etahat
    ('known', None): 'out/CBCconst_δ20_η10_20230810_011115.pkl',  # fixed X̂, learned etahat
}
for seed in [8, 9, 10, 11]:
    pkl_paths |= {
        # eta-known
        # ('unknown', seed): glob(f'out/CBCproj_noise1.0_perm_norm1.0_seed{seed}_2*.pkl')[0],
        # ('topo-14', seed): glob(f'out/CBCproj_noise1.0_perm_norm1.0_seed{seed}_knowntopo14_2*.pkl')[0],
        # ('lines-14', seed): glob(f'out/CBCproj_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],

        # default: δ=20
        ('unknown', seed): glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_2*.pkl')[0],
        # ('topo-14', seed): glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knowntopo14_topochange_2*.pkl')[0],
        ('topo-14', seed): glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knowntopo14_2*.pkl')[0],
        ('lines-14', seed): glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],

        # (r'η* known', seed): glob(f'out/CBCproj_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        # (r'δ=1', seed): glob(f'out/CBCproj_δ1_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        # (r'δ=20', seed): glob(f'out/CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        # (r'δ=100', seed): glob(f'out/CBCproj_δ100_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        # (r'δ=500', seed): glob(f'out/CBCproj_δ500_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
    }

pkls = {}
for (name, seed), pkl_path in pkl_paths.items():
    pkl = load_pkl(pkl_path)
    pkls[(name, seed)] = pkl
    print(f'{name: <15} {str(seed): <4}', pkl.keys())

In [ ]:
def check_consistency(data: dict):
    v = data['vs']
    qc = data['qcs']
    u = qc[1:] - qc[:-1]  # u[t] = u(t) = q^c(t+1) - q^c(t)
    Δv = v[1:] - v[:-1]  # Δv[t] = v(t+1) - v(t)
    count_inconsistent = []
    for t in sorted(data['params'].keys()):
        if data['config']['δ'] == 0:
            X̂ = data['params'][t]
            etahat = 8.65
        else:
            X̂, etahat = data['params'][t]
        vpar_hat = v[1:t+1] - qc[1:t+1] @ X̂
        w_hat = Δv[:t] - u[:t] @ X̂
        consistent = (
            (Vpar_min - 0.05 <= vpar_hat).all(axis=1)
            | (vpar_hat <= Vpar_max + 0.05).all(axis=1)
            | (np.max(np.abs(w_hat), axis=1) <= etahat)
        )
        num_inconsistent = t - consistent.sum()
        count_inconsistent.append(num_inconsistent)
    return count_inconsistent

In [ ]:
count_inconsistent = {}
for (name, seed), data in tqdm(pkls.items()):
    count_inconsistent[(name, seed)] = check_consistency(data)

In [ ]:
fig, axs = plt.subplots(1, 4, sharey=True, figsize=(15, 4))
for (name, seed), counts in count_inconsistent.items():
    if seed is None:
        continue
    ax = axs[seed % 8]
    ax.plot(np.arange(len(counts)) * 100, counts, label=f'{name}')
    ax.legend(loc='upper left')
for ax, seed in zip(axs, [8, 9, 10, 11]):
    ax.set(xlabel='time $t$', title=f'seed {seed}')
    if seed == 8:
        ax.set(ylabel='# of inconsistent data points')

# savefig(fig, plots_dir=plots_dir, filename='violations.png')